In [68]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import Convolution1D, MaxPooling1D
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
 
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
 
import pandas as pd
import MeCab
import numpy as np
import pickle
 
# 適当なデータを用意してください
df = pd.read_csv('data/book.csv', header=None)

def only_str(word):
    if type(word) is word:
        return word
    else:
        pass
#読み込んだcsvから長さ読み込んでもろもろの処理をするメソッドを書く
def init_train():
    print('Loading data')
    Xs = []
    Ys = []
    voc = {}
    maxlen = 0
    maxwords = 0
    buff = set()
    TARGET_DIR = 'data/txt/*'
    idx_name = {}
    print(str(df[3].values))
    for i,filename in enumerate(df[3].str.strip()):
        idx_name[i] = filename
    for line in df[2].str.strip():
        print(only_str(line))
        try:
            a = list(only_str(line))
        except:
            pass
    #行の最大文字数
        maxlen = max(maxlen, len(a))
        [buff.add(w) for w in a]
    #行数
    maxwords = len(buff)
    #限界にMAX,それぞれの項目に数値を入れている
    voc[maxwords] = '___MAX___'
    voc['___META_MAXWORD___'] = maxwords
    voc['___META_MAXLEN___'] = maxlen
    print("maxwords %d"%maxwords)
    print("maxlen %d"%maxlen)
    print("idx name len %d"%len(idx_name))
    for i,filename in enumerate(df[3]):
        for line in set(filter(lambda x:x!='',df[2])): 
            X = [maxwords]*maxlen
            #余分な空白を消す
            line = line.strip()
            for idx, ch in enumerate(list(line)):
            #辞書vocのch番目が殻だったらvocの長さを代入
                if voc.get(ch) == None:
                    voc[ch] = len(voc)
                convert = voc[ch]
                X[idx] = convert
                #Xs,Ysそれぞれにvocの長さ,倍率を追加している
        Xs.append(X)
        y = [0.]*len(idx_name)
        y[i] = 1.
        Ys.append(y)
    X_train, X_test, y_train, y_test = train_test_split( Xs, Ys, test_size=0.1, random_state=42)
    print(voc)
    open('vod.pkl', 'wb').write(pickle.dumps(voc))
    open('idx_name.pkl', 'wb').write(pickle.dumps(idx_name))
    sequence_length = maxlen
    vocabulary_size = maxwords
    embedding_dim   = 256*1
    filter_sizes    = [3,4,5,1,2]
    num_filters     = 512*1
    drop            = 0.5
    
    nb_epoch   = 10
    batch_size = 30
    return sequence_length, embedding_dim, filter_sizes, vocabulary_size, num_filters, drop, idx_name, \
    X_train, X_test, y_train, y_test, batch_size, nb_epoch, Xs, Ys
    
# テキストを分かち書きして返す
def tokenize(text):
    wakati = MeCab.Tagger('-O wakati')
    text = str(text).strip()
    #print(text)
    #encoded_text = text.encode('utf-8')
    return wakati.parse(text)
 
tokenized_text_list = [tokenize(texts) for texts in df[2]]
 
max_features = 5000
maxlen = 600
batch_size = 32
embedding_dim = 50
filter_length = 3
hidden_dims = 250
nb_epoch = 5
 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_text_list)
seq = tokenizer.texts_to_sequences(tokenized_text_list)
X = sequence.pad_sequences(seq, maxlen=maxlen)
Y = str(df[2].sample(n=10)).strip()

#Y = str(df[2]).strip()
 
# モデルを返すメソッド
def build_model():
    print('Build model...')
    model = Sequential()
 
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    model.add(Embedding(max_features,
                        embedding_dims,
                        input_length=maxlen,
                        dropout=0.2))

    # we add a Convolution1D, which will learn nb_filter
    # word group filters of size filter_length:
    model.add(Convolution1D(nb_filter=nb_filter,
                            filter_length=filter_length,
                            border_mode='valid',
                            activation='relu',
                            subsample_length=1))
    # we use max pooling:
    model.add(MaxPooling1D(pool_length=model.output_shape[1]))
 
    # We flatten the output of the conv layer,
    # so that we can add a vanilla dense layer:
    model.add(Flatten())
 
    # We add a vanilla hidden layer:
    model.add(Dense(hidden_dims))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
 
    # We project onto a single unit output layer, and squash it with a sigmoid:
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    adam = Adam()
    model.compile(loss='poisson',
                  optimizer=adam,
                  metrics=['accuracy'])
    return model

def train():
    sequence_length, embedding_dim, filter_sizes, vocabulary_size, num_filters, drop, idx_name, \
    X_train, X_test, y_train, y_test, batch_size, nb_epoch, Xs, Ys = init_train()
    model = build_model(sequence_length=sequence_length, \
    filter_sizes=filter_sizes, \
    embedding_dim=embedding_dim, \
    vocabulary_size=vocabulary_size, \
    num_filters=num_filters, \
    drop=drop, \
    idx_name=idx_name)
    model.fit(Xs, Ys, batch_size=batch_size, nb_epoch=nb_epoch, verbose=1, validation_data=(X_test, y_test))
    model.save('cnn_text_clsfic_all.model')
    

train()
voc = pickle.loads(open('vod.pkl', 'rb').read())
maxlen = voc['___META_MAXLEN___']
maxwords = voc['___META_MAXWORD___']
idx_name = pickle.loads(open('idx_name.pkl', 'rb').read())
model = load_model('cnn_text_clsfic.model')
for line in sys.stdin:
    line = line.strip()
    buff = [maxwords]*maxlen
for i, ch in enumerate(line):
    if voc.get(ch) is None:
        buff[i] = maxwords
    else:
        buff[i] = voc.get(ch)
results = model.predict(np.array([buff]), verbose=0)
preds = results#np.log(results) / 1.0
exp_preds = np.exp(preds)
preds = exp_preds / np.sum(exp_preds)
for result in results:
    logsoftmax = np.log(result)
    max_ent = list(sorted([(i,e) for i,e in enumerate(list(logsoftmax))], key=lambda x:x[1]*-1))[:10]
    _, mini = min(max_ent, key=lambda x:x[1])
    _, maxi = max(max_ent, key=lambda x:x[1])
    base = maxi - mini
    for ent in max_ent:
        id, prob = ent
        prob = (prob - mini)/base
    if int(float(prob)*100) == 0: 
        prob += .01
        #print(mini)
        #print(prob)
    print(idx_name.get(id).split('/').pop().split('.').pop(0), "%d"%(int(float(prob)*100)) + "%")

Loading data
['title' 'title' 'ｏｖｅｒ\u3000ｔｈｅ\u3000ｅｄｇｅ | ジャンル,ミステリ | ハヤカワ・オンライン' ...,
 'サルたちの狂宴 上 | ジャンル,伝記／評論 | ハヤカワ・オンライン' '甦る男 | 受賞作品 | ハヤカワ・オンライン'
 '人生は２０代で決まる | ジャンル,ノンフィクション | ハヤカワ・オンライン']
None


UnboundLocalError: local variable 'a' referenced before assignment